In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# .envファイルから環境変数を読み込む
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1",
)

In [21]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware, HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.types import Command

def get_weather(city: str) -> str:
    """指定した都市の天気を取得します"""
    return f"It's always sunny in {city}!"

def calculator_tool(n1:int, n2:int) -> int:
    """与えられた数字同士を加算します"""
    return n1 + n2

# https://github.com/langchain-ai/docs/blob/main/src/oss/langchain/human-in-the-loop.mdx で修正
def create_new_agent():
    return create_agent(
        model=llm,
        tools=[get_weather, calculator_tool],
        middleware=[
            HumanInTheLoopMiddleware(
                interrupt_on={
                    "calculator_tool": True,  # All decisions (approve, edit, reject) allowed
                    "execute_sql": {"allowed_decisions": ["approve", "reject"]},  # No editing allowed
                    # Safe operation, no approval needed
                    "get_weather": False,
                },
                description_prefix="ツール実行の承認待ち",
            ),
        ],
        checkpointer=InMemorySaver(),
    )


In [22]:
agent = create_new_agent()

"""エージェントのテスト実行"""
print("=== Human-in-the-Loop Agent Test ===\n")

# テストケース1: get_weather (承認不要)
print("テスト1: get_weather (承認不要)")
print("入力: '東京の天気を教えて'")

config = {"configurable": {"thread_id": "test-1"}}

try:
    response = agent.invoke(
        {"messages": [HumanMessage(content="東京の天気を教えて")]},
        config=config
    )
    print(f"結果: {response['messages'][-1].content}\n")
except Exception as e:
    print(f"エラー: {e}\n")

=== Human-in-the-Loop Agent Test ===

テスト1: get_weather (承認不要)
入力: '東京の天気を教えて'
結果: 東京の天気は「いつも晴れ」です！

他に知りたい都市や、今日の気温などもお調べできますので、ご希望があれば教えてください。



In [26]:
agent = create_new_agent()

# テストケース2: calculator_tool (承認必要)
print("テスト2: calculator_tool (承認必要)")
print("入力: '5と3を足して'")

config = {"configurable": {"thread_id": "test-2"}}

try:
    # まずは中断するまで実行
    response = agent.invoke(
        {"messages": [HumanMessage(content="5と3を足して")]},
        config=config
    )
    print(f"中断状態での応答: {response}")
    
    # 状態を確認
    state = agent.get_state(config)
    print(f"エージェントの状態: {state}")
    
    if state.next:
        print("エージェントは中断されています。承認が必要です。")
        
        # 承認して続行
        print("承認して続行...")
        response = agent.invoke(
            Command(resume={"decisions": [{"type": "approve"}]}),
            config=config
        )
        print(f"承認後の結果: {response['messages'][-1].content}")
    
except Exception as e:
    print(f"エラー: {e}\n")

テスト2: calculator_tool (承認必要)
入力: '5と3を足して'
中断状態での応答: {'messages': [HumanMessage(content='5と3を足して', additional_kwargs={}, response_metadata={}, id='f1dafb41-a4ac-44ff-a8a7-0469c4432da9'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 87, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_433e8c8649', 'id': 'chatcmpl-CgKVvgXQiale7X2BPdUxBGgP3veat', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--b1c03c35-daae-4fcd-a67d-cc995b7b6523-0', tool_calls=[{'name': 'calculator_tool', 'args': {'n1': 5, 'n2': 3}, 'id': 'call_BQA8Pe4gq5VkPnvlzul97KDT', 'type': 'tool_call'}], usage_metadata={'input_tokens': 87,

In [27]:
agent = create_new_agent()

# テストケース3: calculator_tool (承認拒否)
print("テスト3: calculator_tool (承認拒否)")
print("入力: '5と3を足して'")

config = {"configurable": {"thread_id": "test-3"}}

try:
    # まずは中断するまで実行
    response = agent.invoke(
        {"messages": [HumanMessage(content="5と3を足して")]},
        config=config
    )
    print(f"中断状態での応答: {response}")
    
    # 状態を確認
    state = agent.get_state(config)
    print(f"エージェントの状態: {state}")
    
    if state.next:
        print("エージェントは中断されています。承認が必要です。")
        
        # 承認して続行
        print("承認して続行...")
        response = agent.invoke(
            Command(resume={"decisions": [{"type": "reject"}]}),
            config=config
        )
        print(f"承認後の結果: {response['messages'][-1].content}")
    
except Exception as e:
    print(f"エラー: {e}\n")

テスト3: calculator_tool (承認拒否)
入力: '5と3を足して'
中断状態での応答: {'messages': [HumanMessage(content='5と3を足して', additional_kwargs={}, response_metadata={}, id='3deccb5c-a0b6-46a8-961f-434d604d6a83'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 87, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_433e8c8649', 'id': 'chatcmpl-CgKW707pDWJrX2B9SCAd36suTGkkW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--3764e20c-cc06-4ade-ab0f-05257c0133a4-0', tool_calls=[{'name': 'calculator_tool', 'args': {'n1': 5, 'n2': 3}, 'id': 'call_6Ej1WvHI6K0ZHnwc2a27627x', 'type': 'tool_call'}], usage_metadata={'input_tokens': 87,